In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import time

import os
import numpy as np
import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, GRU,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
import pandas as pd

In [9]:
# get texts data
category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)

In [10]:
train_texts = train_df_sample.values
label_list = to_categorical(train_df_sample.category)

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values
# print(len(test_texts), test_texts[0]) 

In [11]:
# # process some data
train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])
train_texts = train_texts_list

In [12]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec_20180424.model")
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)

In [13]:
test_texts_list = list(test_texts)
textss = np.append(train_texts , test_texts_list)

In [17]:
tokenizer1 = Tokenizer(num_words = 8000)
tokenizer1.fit_on_texts(textss)
sequences = tokenizer1.texts_to_sequences(textss)  # word index list
max_doc_word_length = 200
sequences1 = pad_sequences(sequences, maxlen=max_doc_word_length, padding='pre')
word_index = tokenizer1.word_counts
print("Found %s unique tokens" % len(tokenizer1.word_counts))

Found 249953 unique tokens


In [18]:

# del sequences1,sequences
# sequences1[9000]

In [19]:
print( len(sequences), tokenizer1.num_words) 

10000 8000


In [89]:
vocab_size = len(word_index) + 1
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 250))
for word, i in word_index.items():
    if word in wvv_keys_list:
        embedding_vector = answer[word]
#         if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.zeros((1, 250))

In [ ]:
# del answer

In [90]:
embedding_layer = Embedding(input_dim= embedding_matrix.shape[0],
                            output_dim= embedding_matrix.shape[1],
                            weights=[embedding_matrix], 
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(GRU(16))    
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 250)         62488500  
_________________________________________________________________
gru_2 (GRU)                  (None, 16)                12816     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               1700      
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1010      
Total params: 62,514,126
Trainable params: 25,626
Non-trainable params: 62,488,500
_________________________________________________________________


In [ ]:
history = model.fit(x = sequences1, y = label_list, 
                    validation_split=0.1, 
                    batch_size= 1620,
                    epochs = 200, verbose = 1)

In [ ]:
# model.save('my_model.h5') 

In [ ]:
# evaluate the model
loss_accuracy = model.evaluate(sequences1[0:100], label_list[0:100], verbose=1)
print(type(loss_accuracy), loss_accuracy)

In [ ]:
# test_tokenizer = Tokenizer()
# tokenizer.fit_on_texts(test_texts)
# test_sequences = tokenizer.texts_to_sequences(test_texts)
# test_sequences1 = pad_sequences(test_sequences, maxlen=max_doc_word_length, padding='post')

In [ ]:
print(len(sequences1))
test_sequences1 = sequences1[9001:10001]

In [ ]:
predict_res = model.predict(test_sequences1, batch_size= 32, verbose=0)
print(len(predict_res), predict_res)

final_res = []
for pre_res in predict_res:
    final_res.append(np.argmax(pre_res))
print(final_res)

In [ ]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "server_result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,category" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(value) + '\n')
        ids += 1